In [1]:
from nba_api.stats.endpoints import leaguedashteamstats, scoreboard, leaguegamefinder
from nba_api.stats.static import teams
import requests
import json
import pandas as pd
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
cavs = 1610612739
clippers = 1610612746
knicks = 1610612752
celtics = 1610612738
hawks = 1610612737
nets = 1610612751
hornets = 1610612766
bulls = 1610612741
mavericks = 1610612742
nuggets = 1610612743
pistons = 1610612765
warriors = 1610612744
rockets = 1610612745
pacers = 1610612754
lakers = 1610612747
grizzlies = 1610612763
heat = 1610612748
bucks = 1610612749
timberwolves = 1610612750
pelicans = 1610612740
thunder = 1610612760
magic = 1610612753
sixers = 1610612755
suns = 1610612756
blazers = 1610612757
kings = 1610612758
spurs = 1610612759
raptors = 1610612761
jazz = 1610612762
wizards = 1610612764

league_ids = [cavs, clippers, knicks, celtics, hawks, nets, hornets, bulls, mavericks, nuggets, pistons, warriors, rockets,
             pacers, lakers, grizzlies, heat, bucks, timberwolves, pelicans, thunder, magic, sixers, suns, blazers, kings,
             spurs, raptors, jazz, wizards]
no_cavs = [cavs, clippers, knicks, celtics, hawks, nets, hornets, bulls, mavericks, nuggets, pistons, warriors, rockets,
          pacers, lakers, grizzlies, heat, bucks, timberwolves, pelicans, thunder, magic, sixers, suns, blazers, kings,
          spurs, raptors, jazz, wizards]

In [3]:
def combine_team_games(df, keep_method='home'):
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_AWAY', '_HOME'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_AWAY != joined.TEAM_ID_HOME]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_AWAY.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_AWAY.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_AWAY == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_AWAY == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
#     display(joined)
    return result

In [4]:
result = leaguegamefinder.LeagueGameFinder()
all_games = result.get_data_frames()[0]

def get_dfs(team_id):
    seasons = ['2022-23', '2021-22', '2020-21', '2019-20']
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable='2023-24')
    all_team_games = gamefinder.get_data_frames()[0]
    
    for i in seasons:
        new_gf = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id, season_nullable=i)
        new_games = new_gf.get_data_frames()[0]
        all_team_games = all_team_games.append(new_games)

    all_team_games.index = range(0, all_team_games.shape[0])
    game_ids = []
    for i in range(all_team_games.shape[0]):
        game_ids.append(all_team_games.loc[i]['GAME_ID'])
        
    rows = []
    for id in game_ids:
        get_games = all_games[all_games.GAME_ID == id]
        new_row = combine_team_games(get_games).iloc[0]
        rows.append(new_row)
    df = pd.DataFrame(rows)
    df_away = df[df['TEAM_ID_AWAY'].isin([team_id])]
    df_home = df[df['TEAM_ID_HOME'].isin([team_id])]
    df_home = df_home.sort_values(by='GAME_DATE', ascending=True)
    df_away = df_away.sort_values(by='GAME_DATE', ascending=True)
    df_away.index = range(0,df_away.shape[0])
    df_home.index = range(0,df_home.shape[0])
    return df_home, df_away

In [5]:
df_home, df_away = get_dfs(knicks)
pd.set_option('max_columns', None)
df_away

/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conc

OptionError: Pattern matched multiple keys

In [6]:
def rolling_averages(group, cols, new_cols):
    rolling_stats = group[cols].rolling(5, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [7]:
cols = ['PTS_AWAY', 'FGM_AWAY',
           'FGA_AWAY', 'FG_PCT_AWAY', 'FG3M_AWAY', 'FG3A_AWAY', 'FG3_PCT_AWAY', 'FTM_AWAY', 'FTA_AWAY',
           'FT_PCT_AWAY', 'OREB_AWAY', 'DREB_AWAY', 'REB_AWAY', 'AST_AWAY',
           'TOV_AWAY', 'PLUS_MINUS_AWAY', 'PTS_HOME', 'FGM_HOME', 'FGA_HOME',
           'FG_PCT_HOME', 'FG3M_HOME', 'FG3A_HOME', 'FG3_PCT_HOME', 'FTM_HOME', 'FTA_HOME',
           'FT_PCT_HOME', 'OREB_HOME', 'DREB_HOME', 'REB_HOME', 'AST_HOME',
           'TOV_HOME', 'PLUS_MINUS_HOME']
away_cols = ['PTS_AWAY', 'FGM_AWAY',
           'FGA_AWAY', 'FG_PCT_AWAY', 'FG3M_AWAY', 'FG3A_AWAY', 'FG3_PCT_AWAY', 'FTM_AWAY', 'FTA_AWAY',
           'FT_PCT_AWAY', 'OREB_AWAY', 'DREB_AWAY', 'REB_AWAY', 'AST_AWAY',
           'TOV_AWAY', 'PLUS_MINUS_AWAY']
home_cols = ['PTS_HOME', 'FGM_HOME', 'FGA_HOME',
           'FG_PCT_HOME', 'FG3M_HOME', 'FG3A_HOME', 'FG3_PCT_HOME', 'FTM_HOME', 'FTA_HOME',
           'FT_PCT_HOME', 'OREB_HOME', 'DREB_HOME', 'REB_HOME', 'AST_HOME',
           'TOV_HOME', 'PLUS_MINUS_HOME']
preds = ['PTS_AWAY', 'FGM_AWAY',
           'FGA_AWAY', 'FG_PCT_AWAY', 'FG3M_AWAY', 'FG3A_AWAY', 'FG3_PCT_AWAY', 'FTM_AWAY', 'FTA_AWAY',
           'FT_PCT_AWAY', 'OREB_AWAY', 'DREB_AWAY', 'REB_AWAY', 'AST_AWAY',
           'TOV_AWAY', 'PLUS_MINUS_AWAY', 'PTS_HOME', 'FGM_HOME', 'FGA_HOME',
           'FG_PCT_HOME', 'FG3M_HOME', 'FG3A_HOME', 'FG3_PCT_HOME', 'FTM_HOME', 'FTA_HOME',
           'FT_PCT_HOME', 'OREB_HOME', 'DREB_HOME', 'REB_HOME', 'AST_HOME',
           'TOV_HOME', 'PLUS_MINUS_HOME']

new_cols = [f"{c}_rolling" for c in cols]
new_cols_home = [f"{c}_rolling" for c in home_cols]
new_cols_home.append(home_cols[-1])
new_cols_away = [f"{c}_rolling" for c in away_cols]
new_cols_away.append(away_cols[-1])

pred_cols = new_cols_away+new_cols_home

preds = [f"{c}_rolling" for c in preds]

def add_new_cols(team_id):
    df_home, df_away = get_dfs(team_id)
    
    df_home = rolling_averages(df_home, cols, new_cols)
    df_away = rolling_averages(df_away, cols, new_cols)
    df_home.index = range(0,df_home.shape[0])
    df_away.index = range(0,df_away.shape[0])
    return df_home, df_away

In [8]:
def df_with_target(team_id):
    df_home, df_away = add_new_cols(team_id)
    df_home["W"] = (df_home["WL_HOME"] == "W").astype("int")
    df_away["W"] = (df_away["WL_AWAY"] == "W").astype("int")
    df_home.index = range(0,df_home.shape[0])
    df_away.index = range(0,df_away.shape[0])
    return df_home, df_away

In [9]:
df1, df2 = df_with_target(knicks)
df1

/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conc

,SEASON_ID,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_ID,GAME_DATE,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,...,FTM_HOME_rolling,FTA_HOME_rolling,FT_PCT_HOME_rolling,OREB_HOME_rolling,DREB_HOME_rolling,REB_HOME_rolling,AST_HOME_rolling,TOV_HOME_rolling,PLUS_MINUS_HOME_rolling,W
0,22019,1610612765,DET,Detroit Pistons,0021900103,2019-11-06,DET vs. NYK,W,240,122,...,14.0,21.2,0.6688,9.6,36.0,45.6,21.4,15.8,-4.4,0
1,22019,1610612742,DAL,Dallas Mavericks,0021900122,2019-11-08,DAL vs. NYK,L,241,102,...,14.0,21.6,0.6558,9.0,31.6,40.6,21.2,15.4,-9.4,1
2,22019,1610612741,CHI,Chicago Bulls,0021900149,2019-11-12,CHI vs. NYK,W,241,120,...,15.0,24.2,0.6276,10.8,33.2,44.0,20.2,15.0,-6.8,0
3,22019,1610612755,PHI,Philadelphia 76ers,0021900202,2019-11-20,PHI vs. NYK,W,240,109,...,15.8,24.8,0.6424,11.6,32.2,43.8,22.2,13.4,-9.6,0
4,22019,1610612761,TOR,Toronto Raptors,0021900258,2019-11-27,TOR vs. NYK,W,240,126,...,15.2,23.8,0.6490,11.0,32.4,43.4,21.8,14.0,-8.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,22023,1610612748,MIA,Miami Heat,0022301096,2024-04-02,MIA vs. NYK,W,241,109,...,13.2,16.2,0.8148,12.2,31.6,43.8,27.4,11.0,8.2,0
197,22023,1610612741,CHI,Chicago Bulls,0022301119,2024-04-05,CHI vs. NYK,W,239,108,...,14.0,17.2,0.8142,11.6,27.8,39.4,28.6,11.0,4.8,0
198,22023,1610612749,MIL,Milwaukee Bucks,0022301139,2024-04-07,MIL vs. NYK,L,239,109,...,14.2,18.0,0.7942,10.6,27.6,38.2,27.2,12.4,1.8,1
199,22023,1610612741,CHI,Chicago Bulls,0022301149,2024-04-09,CHI vs. NYK,L,241,117,...,15.2,19.0,0.8048,12.4,27.8,40.2,26.6,11.6,7.0,1


In [10]:
#combines home games of all teams and away games of all teams
all_home, all_away = df_with_target(cavs)
    
for team in no_cavs:
    df_home, df_away = df_with_target(team)
    all_home = all_home.append(df_home)
    all_away = all_away.append(df_away)
    
all_home.index = range(0,all_home.shape[0])
all_away.index = range(0,all_away.shape[0])
all_home = all_home.dropna()
all_away = all_away.dropna()
        
all_home

/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conc

,SEASON_ID,TEAM_ID_AWAY,TEAM_ABBREVIATION_AWAY,TEAM_NAME_AWAY,GAME_ID,GAME_DATE,MATCHUP_AWAY,WL_AWAY,MIN_AWAY,PTS_AWAY,...,FTM_HOME_rolling,FTA_HOME_rolling,FT_PCT_HOME_rolling,OREB_HOME_rolling,DREB_HOME_rolling,REB_HOME_rolling,AST_HOME_rolling,TOV_HOME_rolling,PLUS_MINUS_HOME_rolling,W
0,22019,1610612764,WAS,Washington Wizards,0021900118,2019-11-08,WAS vs. CLE,L,239,100,...,14.4,18.8,0.7628,8.2,37.6,45.8,17.4,17.4,-16.6,1
1,22019,1610612752,NYK,New York Knicks,0021900136,2019-11-10,NYK vs. CLE,L,239,87,...,15.6,19.8,0.7768,10.0,37.4,47.4,17.6,16.4,-13.2,1
2,22019,1610612755,PHI,Philadelphia 76ers,0021900147,2019-11-12,PHI vs. CLE,W,240,98,...,18.6,22.4,0.8130,9.8,38.2,48.0,19.4,13.8,0.2,0
3,22019,1610612752,NYK,New York Knicks,0021900189,2019-11-18,NYK vs. CLE,W,241,123,...,19.8,23.4,0.8396,9.6,36.6,46.2,18.0,13.2,1.8,0
4,22019,1610612748,MIA,Miami Heat,0021900206,2019-11-20,MIA vs. CLE,W,238,124,...,21.0,27.0,0.7796,9.6,35.2,44.8,17.8,14.8,1.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6587,22023,1610612741,CHI,Chicago Bulls,0022300971,2024-03-16,CHI vs. WAS,W,240,127,...,14.6,19.0,0.7650,7.2,29.2,36.4,29.0,13.0,-12.6,0
6588,22023,1610612741,CHI,Chicago Bulls,0022301037,2024-03-25,CHI vs. WAS,L,240,105,...,15.0,20.4,0.7366,7.2,27.8,35.0,28.6,14.2,-13.4,1
6589,22023,1610612761,TOR,Toronto Raptors,0022301137,2024-04-07,TOR vs. WAS,W,241,130,...,14.2,19.4,0.7380,8.0,28.8,36.8,29.8,13.8,-10.6,0
6590,22023,1610612750,MIN,Minnesota Timberwolves,0022301152,2024-04-09,MIN vs. WAS,W,239,130,...,15.4,21.0,0.7392,8.4,29.0,37.4,28.6,13.8,-12.6,0


In [13]:
model = GaussianNB()

home_train, home_test = train_test_split(all_home, test_size=0.25)
model_home = model.fit(home_train[pred_cols], home_train["W"])
preds = model.predict(home_test[pred_cols])
combined_home = pd.DataFrame(dict(actual=home_test["W"], predicted=preds), index=home_test.index)
accuracy_home = accuracy_score(home_test["W"], preds)
precision_home = precision_score(home_test["W"], preds)

away_train, away_test = train_test_split(all_away, test_size=0.25)
model_away = model.fit(away_train[pred_cols], away_train["W"])
preds = model.predict(away_test[pred_cols])
combined_away = pd.DataFrame(dict(actual=away_test["W"], predicted=preds), index=away_test.index)
accuracy_away = accuracy_score(away_test["W"], preds)
precision_away = precision_score(away_test["W"], preds)

print("Home")
print("Accuracy: " + str(accuracy_home))
print("Precision: " + str(precision_home))
print()
print("Away")
print("Accuracy: " + str(accuracy_away))
print("Precision: " + str(precision_away))

combined_home

Home
Accuracy: 0.962378640776699
Precision: 0.9504685408299867

Away
Accuracy: 0.9450483091787439
Precision: 0.9457700650759219


,actual,predicted
4452,1,1
4200,0,0
2342,1,1
5619,1,1
1474,0,0
...,...,...
6091,1,1
1346,1,1
1978,0,1
6362,1,1


In [14]:
def predictor(away_id, home_id):
    df_home, df_away1 = df_with_target(home_id)
    df_home1, df_away = df_with_target(away_id)
    
    home_stats = list(df_home.loc[:, 'PTS_HOME_rolling':'PLUS_MINUS_HOME_rolling'].iloc[-1])
    home_pm = df_home['PLUS_MINUS_HOME'].iloc[-1]
    home_stats.append(home_pm)
    away_stats = list(df_away.loc[:, 'PTS_AWAY_rolling':'PLUS_MINUS_AWAY_rolling'].iloc[-1])
    away_pm = df_away['PLUS_MINUS_AWAY'].iloc[-1]
    away_stats.append(away_pm)
    new_stats = [away_stats+home_stats]
        
    df_predict = pd.DataFrame(new_stats, columns=pred_cols)
    df_predict['W'] = model_away.predict(new_stats)
    
    if df_predict['W'].iloc[0] == 1:
        result = "win"
    else: 
        result = "lose"
        
    print(f"The away team will {result} against the home team.")

In [15]:
predictor(sixers, knicks)
predictor(magic, cavs)
predictor(lakers, nuggets)

/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conc

The away team will lose against the home team.


/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conc

The away team will win against the home team.


/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_team_games = all_team_games.append(new_games)
/var/folders/5f/fnrjpg_s0xb7q4bkll3k_xq80000gn/T/ipykernel_9222/216743889.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conc

The away team will lose against the home team.


/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
